In [2]:
using Polymake

polymake version 4.11
Copyright (c) 1997-2023
Ewgenij Gawrilow, Michael Joswig, and the polymake team
Technische Universität Berlin, Germany
https://polymake.org

This is free software licensed under GPL; see the source for copying conditions.
There is NO warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [17]:
#This file appends the information of #interior points to our Smooth Polygon Database, which we receive from Balletti

# Smooth_Polytopes_DB is a file of Smooth 2-polytopes after parsing, from Balletti
# Each line takes the form
# 4: 0 0 0 1 1 1 1 0
# The first integer is the number of vertices, subsequent pairs represent vertices, given in clockwise order
filename = "Smooth_Polytopes_DB.txt"

# Open the file
open(filename) do file
    for line in eachline(file)
        # Initialize an empty matrix to store the vectors
        vertices_matrix = Vector{Int}[]
        # Split the line into parts using ": " as the delimiter
        parts = split(line, ": ")
        
        num_vertices = parse(Int, parts[1])
        integers_str = split(parts[2], ' ')
        if length(integers_str) >= num_vertices * 2
            # Extract the remaining integers and convert them to vectors
            vertices = [[1, parse(Int, x), parse(Int, y)] for (x, y) in zip(integers_str[1:2:end], integers_str[2:2:end])]
            
            # Append the vectors to the matrix
            append!(vertices_matrix, vertices)
        end
        
        vertices_array = permutedims(hcat(vertices_matrix...)) #Types conversion that I don't understand 
        #Process the vertices into a polytope, and write out its interior lattice points
        new_polygon = polytope.Polytope(POINTS=vertices_array)
        open("Smooth_Polygon_DB.txt", "a") do file
            if new_polygon.N_LATTICE_POINTS <= 30 #Check if the polytope isn't too large
                #Now output 
                println(file, num_vertices, ':', new_polygon.N_INTERIOR_LATTICE_POINTS, ": ", join([join(row[2:end], ' ') for row in vertices_matrix], ' '))
            end
        end
    end
end